In [27]:
#Library imports
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter

In [57]:
#importing stock list
stocks = pd.read_csv(r"\Users\felix\Documents\Python Stuff\Algorithmic Trading\sp_500_stocks.csv")
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [29]:
#API Call tryout
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SA( GNACSESTMEG/K BLENA DTQ)RLLAO', 'calculationPrice': 'tops', 'open': None, 'openTime': None, 'openSource': 'liafocfi', 'close': None, 'closeTime': None, 'closeSource': 'liifaofc', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 128.56, 'latestSource': 'IEX real time price', 'latestTime': '2:47:48 PM', 'latestUpdate': 1700938055711, 'latestVolume': None, 'iexRealtimePrice': 130.05, 'iexRealtimeSize': 313, 'iexLastUpdated': 1698061899650, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 133.06, 'previousVolume': 80600384, 'change': -1.51, 'changePercent': -0.01213, 'volume': None, 'iexMarketPercent': 0.013637024928518152, 'iexVolume': 798835, 'avgTotalVolume': 95606672, 'iex

In [30]:
#parsing
price = data["latestPrice"]
marketCap = data["marketCap"]

In [32]:
#adding stocks data to a pandas
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
final_Dataframe = pd.DataFrame(columns = my_columns)

In [13]:
#more adding
final_Dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            marketCap,
            "N/A"            
        ],
          index = my_columns
            ),
  
    ignore_index=True    

)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,130.02,2196176209066,N/A


In [19]:
#Looping but very slow because of API request
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks["Ticker"]:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    
    pd.Series(
    [
        stock,
        data["latestPrice"],
        data["marketCap"],
        "N/A",
    ],
        index = my_columns,
        
    ),
        
     ignore_index=True   
    )

In [44]:
#chopping data into chunks in order to do a Batch API call
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]       

In [70]:
#More Looping
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,135.90,40600506136,N/A
1,AAL,23.37,15176943576,N/A
2,AAP,201.54,13236861181,N/A
3,AAPL,127.59,2191703740578,N/A
4,ABBV,119.87,206050956510,N/A
...,...,...,...,...
500,YUM,121.66,36210475517,N/A
501,ZBH,172.11,35919107107,N/A
502,ZBRA,502.22,26683497902,N/A
503,ZION,58.68,9602948966,N/A


In [81]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio: 1000000


In [84]:
#Number of shares to buy calculation
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy,Number Of Shares to Buy
0,A,135.90,40600506136,N/A,14.0
1,AAL,23.37,15176943576,N/A,84.0
2,AAP,201.54,13236861181,N/A,9.0
3,AAPL,127.59,2191703740578,N/A,15.0
4,ABBV,119.87,206050956510,N/A,16.0
...,...,...,...,...,...
500,YUM,121.66,36210475517,N/A,16.0
501,ZBH,172.11,35919107107,N/A,11.0
502,ZBRA,502.22,26683497902,N/A,3.0
503,ZION,58.68,9602948966,N/A,33.0


In [87]:
#Initializing XLSXWriter
writer = pd.ExcelWriter("recommended_trades.xlsx", engine="xlsxwriter")
final_dataframe.to_excel(writer, sheet_name="Recommended Trades", index=False)


In [93]:
background_color = '#00a0a23'
font_color = '#fffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color':background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
           'num_format': '$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
integer_format = writer.book.add_format(
        {
            'num_format': '0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [97]:
writer.sheets['Recommended Trades'].set_column('B:B',
                                              18,
                                               string_format                                          
                                              )

0

In [122]:
column_formats = {
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20,column_formats[column][1])
writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)


0

In [123]:
writer.save()